In [2]:
import pandas as pd
import yaml
from tqdm.notebook import tqdm
#from langchain.chat_models.gigachat import GigaChat
#from langchain.schema import HumanMessage, SystemMessage
import re

from Gigachat_defs import llm_output, gigachat_stub

Set global variables and read configuration data

In [ ]:
## global variables
config_path = r"configs\config.yml"

# few-shot examples
few_shot = pd.read_excel(r"data\few_shot_2_each.xlsx")

# configs for the model
with open(config_path) as fh:
    read_config = yaml.load(fh, Loader=yaml.FullLoader)
creds = read_config['creds']
model = read_config['model']

# prompt for few-shots
system_prompt_news_path = read_config['system_prompt_news_path']
with open(system_prompt_news_path, 'r', encoding="utf8") as system_prompt_news_path:
    system_prompt_news_template = system_prompt_news_path.read()

# prompt for request
prompt_path = read_config['prompt_path']
with open(prompt_path, 'r', encoding="utf8") as prompt_file:
    prompt_template = prompt_file.read()

# news types
type_of_news_path = read_config['type_of_news_path']
with open(type_of_news_path, 'r', encoding="utf8") as type_of_news_file:
    type_of_news = type_of_news_file.read()

In [ ]:
# Let's create 10 positive and 10 negative prompts where sentiment can be distinguished by sign.  
# Let's create 20 neutral (sign-free) prompts for the remaining cases. Those news types have # symbol before their names.

type_of_news_1 = {f'{i+1}': news for i, news in enumerate(type_of_news.splitlines())}
all_prompts = {}
all_prompts_2 = {}

for i in range(1, len(type_of_news.splitlines())+1):
        
        filter = (few_shot.iloc[:,1]==i).tolist()
        two_shots = few_shot.iloc[filter,0].reset_index(drop=True)
        
        # generate 10 positive and neutral news items each  
        if type_of_news_1[f'{i}'][0] != '#':
            specific_prompt_news_template_pos = system_prompt_news_template.format(
                num_and_sign = '10 positive',
                news_type=type_of_news_1[f'{i}'], 
                News_1=two_shots[0], 
                News_2=two_shots[1], 
                )    
                       
            specific_prompt_news_template_neg = system_prompt_news_template.format(
                num_and_sign = '10 negative',
                news_type=type_of_news_1[f'{i}'], 
                News_1=two_shots[0], 
                News_2=two_shots[1],  
                ) 
            
            all_prompts[f'{i}_pos'] = specific_prompt_news_template_pos
            all_prompts[f'{i}_neg'] = specific_prompt_news_template_neg

        # 20 neutral news items for the remaining cases
        else:
            specific_prompt_news_template = system_prompt_news_template.format(
                num_and_sign = '20',
                news_type=type_of_news_1[f'{i}'][1:], 
                News_1=two_shots[0], 
                News_2=two_shots[1], 
                )    

           
            all_prompts[f'{i}'] = specific_prompt_news_template


Заглушка для гигачата - с первого раза он никогда не работает, нужно один раз его запустить через try-except

In [ ]:
# Function to extract the number before the '_' character
def extract_number(s):
    match = re.match(r'(\d+)', s)
    return match.group(1) if match else s

In [5]:
gigachat_stub(creds, model)()

c:\Programming\venvs\telegram_news\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
Giga generation stopped with reason: length


Росбанк является одним из крупнейших банков в России,


'Росбанк является одним из крупнейших банков в России,'

In [ ]:
# create final df
final_df = pd.DataFrame({'type':[], 'No':[], 'text':[]})

for prompt in tqdm(list(all_prompts.keys())):
    
    type = type_of_news_1[extract_number(prompt)]
    
    output = llm_output(type, 
                        all_prompts[prompt], 
                        prompt_template, 
                        temperature = 1.2)
    output = output.replace('\n\n', '\n')

    # Remove the number at the beginning and split into lines
    output = list(map( lambda x: x[3:], output.splitlines()))
    curr_df = pd.DataFrame({'type':[], 'No':[], 'text':[]})
    curr_df['text'] = output
    curr_df['type'] = type
    curr_df['No'] = [i for i in range(1, len(output)+1)]

    final_df = pd.concat([final_df, curr_df])


  0%|          | 0/54 [00:00<?, ?it/s]

Совместный проект с другой компанией___2
#Изменение бренда компании___2


In [ ]:
# # Create weights by news types

final_df_copy = final_df.copy()

weights = [1, 1, 1, 0.4, 0.8, 0.7, 0.4, 0.2, 0.8, 0.5, 0.8, 1, 0.8, 0.4, 0.3, 0.6, 0.7, 0.5, 1, 0.2, 0.6, 0.8, 0.8, 0.6, 1, 1, 0.8, 0.7, 0.4, 0.2, 0.6]
types_2 = type_of_news.splitlines()
new_column = {f'{types_2[i]}': weights[i] for i in range(len(types_2))}

final_df_copy['impact'] = final_df_copy['type'].map(new_column)
final_df_copy.loc[:,'impact'].value_counts()

impact
1.0    140
0.8    140
0.4     80
0.6     80
0.7     60
0.2     60
0.5     40
0.3     20
Name: count, dtype: int64

In [ ]:
# Remove '#' before news types
final_df_copy['type'] = final_df_copy.loc[:,'type'].apply(lambda x: x.replace('#',''))

# There should be 20 news items for each type
final_df_copy.loc[:,'type'].value_counts()

type
Отчет о финансовых результатах                                           20
Политическое событие                                                     20
Изменение бренда компании                                                20
Дробление / сплит / консолидация акций                                   20
Эмиссия облигаций компании                                               20
Объявление об SPO                                                        20
Объявление об IPO                                                        20
Слияние и поглощение                                                     20
Изменение процентной ставки                                              20
Создание нового проекта / кластера / строительства                       20
Крупный заказ / госзаказ                                                 20
Совместный проект с другой компанией                                     20
Кадровое изменение в топ-менеджменте                                     20
Исключе

In [ ]:
# Save data
final_df_copy.to_excel(r'data\synthetic_news.xlsx')